In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
data = pd.DataFrame()
attributes= pd.DataFrame()

# Define the URL of the site
base_site = "https://kolesa.kz/cars/?"
inner_site = "https://kolesa.kz/a/show/"

# Loop through website pages
for i in range(5000):
 
    #sending a request to the webpage
    response = requests.get(base_site+"page={}".format(i))
    
    # get the HTML from the webpage
    html = response.content
#     print(response.status_code)
    
    # convert the HTML to a Beautiful Soup object
    soup = BeautifulSoup(html, 'lxml')
    
    # Extracting the ID, car_name and price of each car on the webpage
    # Find all div tags on the webpage containing the information we want to scrape
    divs = soup.find_all('div', {'class': 'a-info-side col-right-list'})
    a = [divs.find('a', class_='list-link ddl_product_link') for divs in divs]
    ID = [a['data-product-id'] for a in a]
    car_name = [divs.find('a').string for divs in divs]
    price = [divs.find('span', class_="price").text for divs in divs]
    price = [int(''.join(price.split('                    ')[1].split('\xa0')[:-1])) for price in price]
    
    # Loop through IDs to scrape for additional information
    for k in ID:
#         print(k)
        #send request, get HTML and convert to Beautiful Soap object
        response2 = requests.get(inner_site+"{}".format(k))
        html2 = response2.content
        soup2 = BeautifulSoup(html2, 'lxml')
        h1 = soup2.find('h1', {'class': 'offer__title'})
        
        # Extracting the brand, make and year of each car based on ID
        brand = h1.find('span', itemprop="brand").string
        if h1.find('span', itemprop="name") != None:
            make = h1.find('span', itemprop="name").string
        year = int(h1.find('span', class_="year").string)

        #scrape for additional information
        div = soup2.find('div', {'class': 'offer__parameters'})   
        d={}
        for i in div.find_all('dt'):
            if i.find('span') != None:
                d['{}'.format(i.find('span').string)]=div.find('dt', title=i['title']).parent.find('dd').string.strip()
        d.update({'Марка': brand, 'Модель': make, 'Год выпуска': year})
        data1 = [d]
#         print(data1)
        #store scraped information in a 'attributes' dataframe
        attributes = attributes.append(data1,ignore_index=True,sort=False)
    
    #create new_data dataframe to store car_name, price and ID information and concatinate with master dataframe
    new_data=pd.DataFrame()
    new_data['Авто'] = car_name
    new_data['Цена'] = price
    new_data['ID'] = ID
    data = pd.concat([data,new_data], axis=0, ignore_index=True)
    
#join two dataframes by axis = 1
data = data.join(attributes)

#save dataframe to excel
data.to_excel(r'C:\Users\ecws\Desktop\Kolesa_data.xlsx', index = False)

In [5]:
data.head()

,Авто,Цена,ID,Город,Кузов,"Объем двигателя, л",Коробка передач,Руль,Цвет,Растаможен в Казахстане,Марка,Модель,Год выпуска,Привод,Пробег,VIN
0,Chevrolet Cruze,3100000,103248829,Кызылорда,седан,1.8 (бензин),автомат,слева,белый,Да,Chevrolet,Cruze,2011,NaN,NaN,NaN
1,Mercedes-Benz E 280,2150000,101652686,Талдыкорган,седан,2.8 (бензин),механика,слева,серебристый металлик,Да,Mercedes-Benz,E 280,1994,задний привод,NaN,NaN
2,Chevrolet Aveo,2850000,102804247,Атырау,седан,1.6 (бензин),механика,слева,белый,Да,Chevrolet,Aveo,2013,передний привод,105 000 км,NaN
3,Hyundai Sonata,4000000,98743048,Атырау,седан,2.4 (бензин),типтроник,слева,серебристый металлик,Нет,Hyundai,Sonata,2012,передний привод,135 000 км,NaN
4,Lexus RX 400h,6800000,105381955,Талдыкорган,кроссовер,3.3 (гибрид),автомат,слева,серебристый металлик,Да,Lexus,RX 400h,2006,полный привод,240 000 км,NaN
